# Module I: Infinite square well

Comments can be sent to: Trond S. Ingebrigtsen, trond@ruc.dk. Visit this [link](http://trondingebrigtsen.com) for more Jupyter modules. 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from scipy.integrate import trapz
from matplotlib import rc
rc('animation', html='jshtml')

import warnings
warnings.filterwarnings("ignore")

This notebook is based on D. J. Griffiths' treatment of the infinite square well (Chapter 2) and U. R. Pedersen's Matlab tutorial. After this module you will understand:

    1. Stationary states.
    2. A linear combination of two stationary states as the initial state.
    3. The motion of a general initial state.
    4. Analyzing the motion in terms of expectation values such as <x> and Heisenberg's uncertainty principle.

The module is intended for about **4 hours** of work. You will need to hand it in **next week**. 

# 1. Schr&ouml;dinger's equation in Hartree units

The fundamental laws of physics depend on which unit system we use to describe them. The most standard is SI units. However, 
in quantum mechanics it is sometimes convenient to use a unit system suggested by Hartree where $m$ = $e$ = $\hbar$ = $k$ = 1. Here, $m$ is the electron mass, $e$ is the elementary charge, $\hbar$ is Planck's constant divided by $2\pi$, and $k$ = 1/($4\pi\epsilon_{0})$ is Coulomb's constant. In these units the time-dependent Schr&ouml;dinger equation (SE) becomes

\begin{align}
i\frac{\partial \Psi(x,t)}{\partial t} = -\frac{1}{2}\frac{\partial^{2}\Psi(x,t)}{\partial x^{2}} + V(x)\Psi(x,t),
\end{align}

and the time-independent SE becomes

\begin{align}
 -\frac{1}{2}\frac{\partial^{2}\psi(x)}{\partial x^{2}} + V(x)\psi(x) = E\psi(x).
\end{align}

This unit system is suitable for computer implementation.

# 2. Stationary states

This section will enable you to plot the time evolution of stationary states of the infinite square well in Jupyter. Recall that stationary states have the general form

\begin{align}
\Psi_{n}(x,t) = \psi_{n}(x)e^{-iE_{n}t/\hbar},
\end{align}

where $\psi_{n}(x)$ and $E_{n}$ can be found Chapter 2.2, remember to use Hartree units.
<br><br>
**How to proceed wih this module:**

You will need to insert your own Python code in front of the sentence: **"??? Implement me ???"**. Throughout the code you will also encounter **"Exercise X"** which indicates a modification related to Exercise X of the same section. You are **expected** to document your reasoning/thoughts and findings in each section. Enjoy!

In [2]:
# The stationary state n to plot, the well length a, 
# and the time step dt for animating the wave function. 
n, a, dt = 3, 1, 0.01

# The position range x to plot.
x = np.linspace(0,a,1000)

# Implementation of stationary states. Include the time dependence.
def stationary_state(n,x,t):
    # Energy.
    En = (n*math.pi)**2/(2*a**2) # ??? Implement me ???
    
    # Wave function.
    nth_solution = math.sqrt(2/a) * np.sin((n*math.pi/a)*x) * np.exp(-1j*En*t) # ??? Implement me ???
    return nth_solution

#################################################################################

# For plotting the animation. A detailed understanding is not needed.
fig0, ax0 = plt.subplots(figsize=(6, 4))
ax0.set_xlabel(r'$x$', fontsize=16, fontname="Arial") 
ax0.set_ylabel(r'$\Psi_{n}(x,t)$', fontsize=16, fontname="Arial")
fig0.tight_layout(rect=[0, 0.03, 1, 0.95])
l, = ax0.plot([0,a],[-2*a,2*a]) # Change axis scale here.
plt.close()

# Shows the real part of the stationary solution.
def animate0(i):
    ax0.set_title('$t$ = %f' % (i*dt), fontsize=16, fontname="Arial")
    l.set_data(x, stationary_state(n,x,i*dt).real) # Exercise 1 
    #l.set_data(x, np.conj(stationary_state(n,x,i*dt)) * stationary_state(n,x,i*dt))
    return l,

ani0 = animation.FuncAnimation(fig0, animate0, frames=50, interval=200, repeat=False)
ani0

findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


**Exercise 1:** Change the value of $n = 1, ..., \infty$ and plot the probability density $|\Psi_{n}(x,t)|^{2}$ by modifying the $animate0$ code at "Exercise 1".<br> Is anything different from $\Psi_{n}(x,t)$? If so explain why. 

# 3. An even mixture of two stationary states

This section considers an even mixture of two stationary states as the initial state, i.e., $\Psi(x,0) = \psi_{1}(x)$ + $\psi_{2}(x)$. It is based on **Problem 2.5** in Griffiths. In general, to plot an initial wave function we need to use Fourier's trick 

$c_{n}$ = $\langle \psi_{n}|\Psi \rangle$ = $\int \psi_{n}(x)^{*}\Psi(x,0) dx$,

to determine the expansion coefficients $c_{n}$ in the general solution for the wave function

$\Psi(x,t)$ = $\sum_{n=1}^{\infty}c_{n}\psi_{n}(x)e^{-iE_{n}t/\hbar}$.

We will implement this approach here. Remember that normalization amounts to 

$\langle \Psi|\Psi \rangle$ = $1$.

In [3]:
# Psi(x,0) = psi_1(x) + psi_2(x). Normalized.
def initial_state(x):
    initial_sum = stationary_state(1,x,0) + stationary_state(2,x,0) # Exercise 3. 
    #initial_sum = stationary_state(1,x,0) + stationary_state(2,x,0)*np.exp(1j*math.pi/2)
    
    # Normalization constant for initial_sum.
    A = np.sqrt(trapz(np.conj(initial_sum) * initial_sum,x)) # ??? Implement me ???

    return initial_sum / A

# Calculates a general solution and returns the wave function and cn's.    
def wave_function(x,t,cn,initial_state): 
    # The coefficients.
    cn_array = np.zeros(cn,dtype=np.complex_)
    
    total = 0
    for i in range(1,cn):
        # Fourier's trick. Note the t = 0.
        cn_array[i] = trapz(np.conj(stationary_state(i,x,0)) * initial_state,x) # ??? Implement me ???
 
        # General solution sum.
        total += cn_array[i] * stationary_state(i,x,t) # ??? Implement me ???
       
    return total, cn_array

#################################################################################

# For plotting animation.
fig1, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))
ax1.set_xlabel(r'$x$', fontsize=16, fontname="Arial")
ax1.set_ylabel(r'$|\Psi(x,t)|^{2}$', fontsize=16, fontname="Arial")
ax2.set_xlabel(r'$t$', fontsize=16, fontname="Arial") 
ax2.set_ylabel(r'$\langle x \rangle$', fontsize=16, fontname="Arial")
fig1.tight_layout(rect=[0, 0.03, 1, 0.95])
l1, = ax1.plot([0,a],[0,3.5*a]) # Change axis scale here.
l2, = ax2.plot([0,1],[0,a]) 
plt.close()

# How many basic states to use in the expansion.
cn = 10

time = []; mean = [];
def animate1(i):
    ax1.set_title(r'$t$ = %f' % (i*dt), fontsize=16, fontname="Arial")
    ax2.set_title('t = %f' % (i*dt), fontsize=16, fontname="Arial")
   
    my_wave, ci_array = wave_function(x,i*dt,cn,initial_state(x))
    density = np.conj(my_wave) * my_wave
    l1.set_data(x, density)
    
    # Calculates <x>.
    my_x = trapz(np.conj(my_wave) * x * my_wave,x) # Exercise 2.
    #my_x_sq = trapz(np.conj(my_wave) * x * x * my_wave,x)
    #my_std = my_x_sq - my_x * my_x
    
    time.append(i*dt)
    mean.append(my_x) 
    l2.set_data(time,mean)
    
    return l1,

# Problem 2.5 solution (red curve in the right figure).
omega = math.pi**2 / (2*a**2)
time_solution = np.linspace(0,1,1000);
anwser_x = (a/2)*(1 - (32/(9*math.pi**2))*np.cos(3*omega*time_solution))
ax2.scatter(time_solution,anwser_x,marker='o',s=5,linewidths=0.3, color='red')

ani1 = animation.FuncAnimation(fig1, animate1, frames=100, interval=200, repeat=False)
ani1

Output hidden; open in https://colab.research.google.com to view.

**Exercise 1:** What is your interpretation of the time evolution of $|\Psi(x,t)|^{2}$?

**Exercise 2:** Calculate the expectation values $\langle x \rangle $, $\langle x^{2}\rangle$, and $\sigma_{x}^{2}$ = $\langle x^{2}\rangle - \langle x \rangle^{2}$. 

**Exercise 3:** Try to use the initial state $\Psi(x,0) = \psi_{1}(x)$ + $e^{i\phi}\psi_{2}(x)$ with $\phi = \pi /2$. What does the extra factor amount to?

# 4. A general initial state

In [4]:
# Define your own function here.
def initial_state_general(x):
    function = x*(x**3 * (1-x))**2
     
    # Normalization constant.
    A = np.sqrt(trapz(np.conj(function) * function,x))
    
    return function / A

#################################################################################

# For plotting animation.
fig2, (ax3,ax4) = plt.subplots(1,2,figsize=(8,3))
ax3.set_xlabel(r'$x$', fontsize=16, fontname="Arial") 
ax3.set_ylabel(r'$|\Psi(x,t)|^{2}$', fontsize=16, fontname="Arial")
ax4.set_xlabel(r'$t$', fontsize=16, fontname="Arial") 
ax4.set_ylabel(r'$\langle p \rangle$', fontsize=16, fontname="Arial")
fig2.tight_layout(rect=[0, 0.03, 1, 0.95])
l3, = ax3.plot([0,a],[0,6*a]) # Change axis scale here. 
l4, = ax4.plot([0,1],[-5,5]) 
plt.close()

# Is 100 a good number?
cn = 100

time_p = []; mean_p = [];
def animate2(i):
    ax3.set_title(r'$t$ = %f' % (i*dt), fontsize=16, fontname="Arial")
    ax4.set_title(r'$t$ = %f' % (i*dt), fontsize=16, fontname="Arial")
    
    my_wave, ci_array = wave_function(x,i*dt,cn,initial_state_general(x))
    density = np.conj(my_wave) * my_wave
    l3.set_data(x, density)
    
    # Calculates <p>. 
    derivative_p = -1j * np.gradient(my_wave, x) 
    my_p = trapz(np.conj(my_wave) * derivative_p,x) # Exercise 1.

    # calculates <p^2>, sigma_p.
    onederivative_p = np.gradient(my_wave,x)
    twoderivative_p = np.gradient(onederivative_p,x)
    my_p_sq = -1 * trapz(np.conj(my_wave) * twoderivative_p,x)
    my_var_p = my_p_sq - my_p**2

    # Calculates Heisenberg. 
    my_x = trapz(np.conj(my_wave) * x * my_wave,x)
    my_x_sq = trapz(np.conj(my_wave) * x * x * my_wave,x)
    my_var_x = my_x_sq - my_x * my_x  

    heisenberg = math.sqrt(my_var_x * my_var_p) 

    time_p.append(i*dt)
    mean_p.append(my_p) 
    l4.set_data(time_p, mean_p)
    return l3,

ani3 = animation.FuncAnimation(fig2, animate2, frames=100, interval=200, repeat=False)
ani3

Output hidden; open in https://colab.research.google.com to view.

**Exercise 1:** Choose your own initial state and calculate the expectation values $\langle p \rangle$, $\langle p^{2} \rangle $, and $\sigma_{p}^{2}$. Can you choose *any* old initial state? Describe what happens with your choice for the initial state.

**Exercise 2:** Use your wave function to calculate $\langle x \rangle$, $\langle p \rangle$, etc. Is Heisenberg (not the Breaking Bad Heisenberg) happy? Explain with your **own words** the content of Heisenberg's uncertainty principle.

# 5. Module completed

Congratulations! You will need to save the module and hand it in **next week**. Future suggestions for this module can also be written here.